# day 19

https://adventofcode.com/2019/day/19

In [1]:
import logging
import logging.config
import os

import yaml

In [2]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [3]:
FNAME = os.path.join('data', 'day19.txt')

LOGGER = logging.getLogger('day19')

## part 1

### problem statement:

#### loading data

In [4]:
test_data = []

In [77]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return tuple([int(_) for _ in fp.read().strip().split(",")])

#### function def

In [9]:
import aoc2019 as A

In [10]:
STATIONARY = 0
PULLED = 1

In [12]:
import tqdm.autonotebook as tqdm

In [73]:
import functools

In [114]:
@functools.lru_cache()
def get_grid_val(data, inputs):
    return A.IntcodeComputer(list(data), inputs=list(inputs)).get_output()

In [115]:
@functools.lru_cache()
def q_1(data, N=50):
    grid = {}
    for x in tqdm.trange(N):
        for y in tqdm.trange(N, leave=False):
            grid[x, y] = get_grid_val(data, (x, y))
    LOGGER.info(f'num pulled: {len([v for v in grid.values() if v == PULLED])}')
    return grid

#### tests

In [116]:
# def test_q_1():
#     LOGGER.setLevel(logging.DEBUG)
#     assert q_1(test_data) == True
#     LOGGER.setLevel(logging.INFO)

In [117]:
# test_q_1()

#### answer

In [118]:
grid = q_1(load_data())


2019-12-19 00:47:33,367 INFO     [day19.q_1:7] num pulled: 126


In [119]:
def draw_grid(grid):
    s = ''
    for x in range(50):
        for y in range(50):
            s += '#' if grid[x, y] == PULLED else '.'
        s += '\n'
    return s

print(draw_grid(grid))

#.................................................
..................................................
...#..............................................
....#.............................................
......#...........................................
.......#..........................................
........##........................................
..........#.......................................
...........##.....................................
............##....................................
..............##..................................
...............##.................................
................###...............................
..................##..............................
...................###............................
....................###...........................
......................###.........................
.......................###........................
........................####......................
..........................###..

## part 2

### problem statement:

#### function def

In [120]:
grid = q_1(load_data())

In [121]:
ones = {k for (k, v) in grid.items() if v == 1}

In [122]:
current_top_right = max(ones, key=lambda rec: (rec[1], rec[0]))
current_top_right

(37, 49)

In [123]:
current_bottom_left = max(ones, key=lambda rec: (rec[0], rec[1]))
current_bottom_left

(37, 49)

In [135]:
def q_2(data):
    grid = q_1(data)
    
    # we only need to know the top and bottom edges of the ray, I think
    ones = {k for (k, v) in grid.items() if v == 1}
    
    current_top_right = max(ones, key=lambda rec: (rec[1], rec[0]))
    current_bottom_left = max(ones, key=lambda rec: (rec[0], rec[1]))
    
    top_rights = set([current_top_right])
    bottom_lefts = set([current_bottom_left])
    
    while True:
        # check for next top-right edge point:
        x, y = current_top_right
        LOGGER.info(f'current_top_right = {current_top_right}')
        if get_grid_val(data, (x, y + 1)) == PULLED:
            current_top_right = x, y + 1
            top_rights.add(current_top_right)
        elif get_grid_val(data, (x + 1, y)) == PULLED:
            current_top_right = x + 1, y
            top_rights.add(current_top_right)
        else:
            raise ValueError(f'huh? a hole? (x, y) = {(x, y)})')
        
        # same for next bottom-left edge point:
        x, y = current_bottom_left
        if get_grid_val(data, (x + 1, y)) == PULLED:
            current_bottom_left = x + 1, y
            bottom_lefts.add(current_bottom_left)
        elif get_grid_val(data, (x, y + 1)) == PULLED:
            current_bottom_left = x, y + 1
            bottom_lefts.add(current_bottom_left)
        else:
            raise ValueError(f'huh? a hole? (x, y) = {(x, y)})')
        
        # assuming convexity, any time that there is a top right for which
        # x + 100, y - 100 is a bottom left, the square is in there
        for (x_tr, y_tr) in top_rights:
            if any(((x_tr + 100 <= x_bl)
                    and (y_bl + 100 <= y_tr))
                   for (x_bl, y_bl) in bottom_lefts):
                LOGGER.warning(f'top_right = {x_tr, y_tr}')
                return x_tr, y_tr - 100

#### tests

In [136]:
# def test_q_2():
#     LOGGER.setLevel(logging.DEBUG)
#     assert q_2(test_data) == True
#     LOGGER.setLevel(logging.INFO)

In [137]:
# test_q_2()

#### answer

In [138]:
q_2(load_data())

2019-12-19 00:58:25,178 INFO     [day19.q_2:16] current_top_right = (37, 49)
2019-12-19 00:58:25,268 INFO     [day19.q_2:16] current_top_right = (37, 50)
2019-12-19 00:58:25,391 INFO     [day19.q_2:16] current_top_right = (37, 51)
2019-12-19 00:58:25,461 INFO     [day19.q_2:16] current_top_right = (37, 52)
2019-12-19 00:58:25,563 INFO     [day19.q_2:16] current_top_right = (37, 53)
2019-12-19 00:58:25,629 INFO     [day19.q_2:16] current_top_right = (37, 54)
2019-12-19 00:58:25,722 INFO     [day19.q_2:16] current_top_right = (37, 55)
2019-12-19 00:58:25,786 INFO     [day19.q_2:16] current_top_right = (37, 56)
2019-12-19 00:58:25,909 INFO     [day19.q_2:16] current_top_right = (38, 56)
2019-12-19 00:58:25,971 INFO     [day19.q_2:16] current_top_right = (38, 57)
2019-12-19 00:58:26,106 INFO     [day19.q_2:16] current_top_right = (39, 57)
2019-12-19 00:58:26,199 INFO     [day19.q_2:16] current_top_right = (39, 58)
2019-12-19 00:58:26,265 INFO     [day19.q_2:16] current_top_right = (39, 59)

2019-12-19 00:58:34,821 INFO     [day19.q_2:16] current_top_right = (71, 107)
2019-12-19 00:58:34,932 INFO     [day19.q_2:16] current_top_right = (72, 107)
2019-12-19 00:58:35,047 INFO     [day19.q_2:16] current_top_right = (72, 108)
2019-12-19 00:58:35,120 INFO     [day19.q_2:16] current_top_right = (72, 109)
2019-12-19 00:58:35,259 INFO     [day19.q_2:16] current_top_right = (73, 109)
2019-12-19 00:58:35,531 INFO     [day19.q_2:16] current_top_right = (73, 110)
2019-12-19 00:58:35,631 INFO     [day19.q_2:16] current_top_right = (74, 110)
2019-12-19 00:58:35,724 INFO     [day19.q_2:16] current_top_right = (74, 111)
2019-12-19 00:58:35,790 INFO     [day19.q_2:16] current_top_right = (74, 112)
2019-12-19 00:58:35,912 INFO     [day19.q_2:16] current_top_right = (75, 112)
2019-12-19 00:58:35,980 INFO     [day19.q_2:16] current_top_right = (75, 113)
2019-12-19 00:58:36,125 INFO     [day19.q_2:16] current_top_right = (76, 113)
2019-12-19 00:58:36,191 INFO     [day19.q_2:16] current_top_righ

2019-12-19 00:58:44,208 INFO     [day19.q_2:16] current_top_right = (107, 162)
2019-12-19 00:58:44,326 INFO     [day19.q_2:16] current_top_right = (108, 162)
2019-12-19 00:58:44,418 INFO     [day19.q_2:16] current_top_right = (108, 163)
2019-12-19 00:58:44,481 INFO     [day19.q_2:16] current_top_right = (108, 164)
2019-12-19 00:58:44,602 INFO     [day19.q_2:16] current_top_right = (109, 164)
2019-12-19 00:58:44,665 INFO     [day19.q_2:16] current_top_right = (109, 165)
2019-12-19 00:58:44,786 INFO     [day19.q_2:16] current_top_right = (110, 165)
2019-12-19 00:58:44,857 INFO     [day19.q_2:16] current_top_right = (110, 166)
2019-12-19 00:58:44,949 INFO     [day19.q_2:16] current_top_right = (110, 167)
2019-12-19 00:58:45,042 INFO     [day19.q_2:16] current_top_right = (111, 167)
2019-12-19 00:58:45,135 INFO     [day19.q_2:16] current_top_right = (111, 168)
2019-12-19 00:58:45,254 INFO     [day19.q_2:16] current_top_right = (112, 168)
2019-12-19 00:58:45,318 INFO     [day19.q_2:16] curr

2019-12-19 00:58:52,801 INFO     [day19.q_2:16] current_top_right = (143, 217)
2019-12-19 00:58:52,924 INFO     [day19.q_2:16] current_top_right = (144, 217)
2019-12-19 00:58:52,993 INFO     [day19.q_2:16] current_top_right = (144, 218)
2019-12-19 00:58:53,119 INFO     [day19.q_2:16] current_top_right = (145, 218)
2019-12-19 00:58:53,186 INFO     [day19.q_2:16] current_top_right = (145, 219)
2019-12-19 00:58:53,280 INFO     [day19.q_2:16] current_top_right = (145, 220)
2019-12-19 00:58:53,405 INFO     [day19.q_2:16] current_top_right = (146, 220)
2019-12-19 00:58:53,472 INFO     [day19.q_2:16] current_top_right = (146, 221)
2019-12-19 00:58:53,595 INFO     [day19.q_2:16] current_top_right = (147, 221)
2019-12-19 00:58:53,662 INFO     [day19.q_2:16] current_top_right = (147, 222)
2019-12-19 00:58:53,759 INFO     [day19.q_2:16] current_top_right = (147, 223)
2019-12-19 00:58:53,855 INFO     [day19.q_2:16] current_top_right = (148, 223)
2019-12-19 00:58:53,953 INFO     [day19.q_2:16] curr

2019-12-19 00:59:01,698 INFO     [day19.q_2:16] current_top_right = (180, 271)
2019-12-19 00:59:01,771 INFO     [day19.q_2:16] current_top_right = (180, 272)
2019-12-19 00:59:01,871 INFO     [day19.q_2:16] current_top_right = (180, 273)
2019-12-19 00:59:01,999 INFO     [day19.q_2:16] current_top_right = (181, 273)
2019-12-19 00:59:02,075 INFO     [day19.q_2:16] current_top_right = (181, 274)
2019-12-19 00:59:02,204 INFO     [day19.q_2:16] current_top_right = (182, 274)
2019-12-19 00:59:02,283 INFO     [day19.q_2:16] current_top_right = (182, 275)
2019-12-19 00:59:02,384 INFO     [day19.q_2:16] current_top_right = (182, 276)
2019-12-19 00:59:02,485 INFO     [day19.q_2:16] current_top_right = (183, 276)
2019-12-19 00:59:02,588 INFO     [day19.q_2:16] current_top_right = (183, 277)
2019-12-19 00:59:02,691 INFO     [day19.q_2:16] current_top_right = (184, 277)
2019-12-19 00:59:02,793 INFO     [day19.q_2:16] current_top_right = (184, 278)
2019-12-19 00:59:02,896 INFO     [day19.q_2:16] curr

...

2019-12-19 01:03:56,320 INFO     [day19.q_2:16] current_top_right = (685, 1039)
2019-12-19 01:03:56,750 INFO     [day19.q_2:16] current_top_right = (685, 1040)
2019-12-19 01:03:57,188 INFO     [day19.q_2:16] current_top_right = (686, 1040)
2019-12-19 01:03:57,623 INFO     [day19.q_2:16] current_top_right = (686, 1041)
2019-12-19 01:03:58,057 INFO     [day19.q_2:16] current_top_right = (686, 1042)
2019-12-19 01:03:58,494 INFO     [day19.q_2:16] current_top_right = (687, 1042)
2019-12-19 01:03:58,928 INFO     [day19.q_2:16] current_top_right = (687, 1043)
2019-12-19 01:03:59,369 INFO     [day19.q_2:16] current_top_right = (688, 1043)
2019-12-19 01:03:59,804 INFO     [day19.q_2:16] current_top_right = (688, 1044)
2019-12-19 01:04:00,215 INFO     [day19.q_2:16] current_top_right = (688, 1045)
2019-12-19 01:04:00,679 INFO     [day19.q_2:16] current_top_right = (689, 1045)
2019-12-19 01:04:01,118 INFO     [day19.q_2:16] current_top_right = (689, 1046)
2019-12-19 01:04:01,570 INFO     [day19.

2019-12-19 01:04:37,379 INFO     [day19.q_2:16] current_top_right = (721, 1093)
2019-12-19 01:04:37,853 INFO     [day19.q_2:16] current_top_right = (721, 1094)
2019-12-19 01:04:38,330 INFO     [day19.q_2:16] current_top_right = (721, 1095)
2019-12-19 01:04:38,805 INFO     [day19.q_2:16] current_top_right = (722, 1095)
2019-12-19 01:04:39,283 INFO     [day19.q_2:16] current_top_right = (722, 1096)
2019-12-19 01:04:39,768 INFO     [day19.q_2:16] current_top_right = (723, 1096)
2019-12-19 01:04:40,246 INFO     [day19.q_2:16] current_top_right = (723, 1097)
2019-12-19 01:04:40,703 INFO     [day19.q_2:16] current_top_right = (723, 1098)
2019-12-19 01:04:41,223 INFO     [day19.q_2:16] current_top_right = (724, 1098)
2019-12-19 01:04:41,814 INFO     [day19.q_2:16] current_top_right = (724, 1099)
2019-12-19 01:04:42,300 INFO     [day19.q_2:16] current_top_right = (725, 1099)
2019-12-19 01:04:42,789 INFO     [day19.q_2:16] current_top_right = (725, 1100)
2019-12-19 01:04:43,245 INFO     [day19.

2019-12-19 01:05:22,369 INFO     [day19.q_2:16] current_top_right = (756, 1148)
2019-12-19 01:05:22,885 INFO     [day19.q_2:16] current_top_right = (757, 1148)
2019-12-19 01:05:23,404 INFO     [day19.q_2:16] current_top_right = (757, 1149)
2019-12-19 01:05:23,929 INFO     [day19.q_2:16] current_top_right = (758, 1149)
2019-12-19 01:05:24,452 INFO     [day19.q_2:16] current_top_right = (758, 1150)
2019-12-19 01:05:24,946 INFO     [day19.q_2:16] current_top_right = (758, 1151)
2019-12-19 01:05:25,497 INFO     [day19.q_2:16] current_top_right = (759, 1151)
2019-12-19 01:05:26,017 INFO     [day19.q_2:16] current_top_right = (759, 1152)
2019-12-19 01:05:26,541 INFO     [day19.q_2:16] current_top_right = (760, 1152)
2019-12-19 01:05:27,064 INFO     [day19.q_2:16] current_top_right = (760, 1153)
2019-12-19 01:05:27,558 INFO     [day19.q_2:16] current_top_right = (760, 1154)
2019-12-19 01:05:28,107 INFO     [day19.q_2:16] current_top_right = (761, 1154)
2019-12-19 01:05:28,601 INFO     [day19.

2019-12-19 01:06:16,494 INFO     [day19.q_2:16] current_top_right = (792, 1202)
2019-12-19 01:06:17,082 INFO     [day19.q_2:16] current_top_right = (792, 1203)
2019-12-19 01:06:17,727 INFO     [day19.q_2:16] current_top_right = (793, 1203)
2019-12-19 01:06:18,306 INFO     [day19.q_2:16] current_top_right = (793, 1204)
2019-12-19 01:06:18,941 INFO     [day19.q_2:16] current_top_right = (794, 1204)
2019-12-19 01:06:19,562 INFO     [day19.q_2:16] current_top_right = (794, 1205)
2019-12-19 01:06:20,153 INFO     [day19.q_2:16] current_top_right = (794, 1206)
2019-12-19 01:06:20,970 INFO     [day19.q_2:16] current_top_right = (795, 1206)
2019-12-19 01:06:21,551 INFO     [day19.q_2:16] current_top_right = (795, 1207)
2019-12-19 01:06:22,161 INFO     [day19.q_2:16] current_top_right = (796, 1207)
2019-12-19 01:06:22,699 INFO     [day19.q_2:16] current_top_right = (796, 1208)
2019-12-19 01:06:23,267 INFO     [day19.q_2:16] current_top_right = (796, 1209)
2019-12-19 01:06:23,924 INFO     [day19.

KeyboardInterrupt: 

fin

In [128]:
def readfile(path):
    with open(path) as input_file: 
        return input_file.read().strip()

def params(program, i, count):
    modes = str(program[i] // 100)[::-1]
    result = []
    
    for x in range(count - 1):
        arg = program[i + 1 + x]
        if x >= len(modes) or modes[x] == "0":
            result.append(program[arg])
        elif modes[x] == "2":
            result.append(program[program.offset + arg])
        else:
            result.append(arg)
            
    output = program[i + count]
    if len(modes) >= count and modes[count - 1] == "2":
        output += program.offset
        
    result.append(output) # output
    return result

def add(program, i, **kwargs):
    assert program[i] % 100 == 1
    a, b, out = params(program, i, 3)
    program[out] = a + b
    return i + 4, None
    
def mul(program, i, **kwargs):
    assert program[i] % 100 == 2
    a, b, out = params(program, i, 3)
    program[out] = a * b
    return i + 4, None

def inp(program, i, input_value, **kwargs):
    assert program[i] % 100 == 3
    out, = params(program, i, 1)
    program[out] = input_value
    # print(f"> {program[i]} program[{out}] = {program[out]}")
    return i + 2, None

def out(program, i, **kwargs):
    assert program[i] % 100 == 4
    val, _ = params(program, i, 2)
    # print(f"> {val}")
    return i + 2, val
    
def jit(program, i, **kwargs):
    assert program[i] % 100 == 5
    test, ip, _ = params(program, i, 3)
    if test != 0:
        return ip, None
    return i + 3, None

def jif(program, i, **kwargs):
    assert program[i] % 100 == 6
    test, ip, _ = params(program, i, 3)
    if test == 0:
        return ip, None
    return i + 3, None

def lt(program, i, **kwargs):
    assert program[i] % 100 == 7
    a, b, out = params(program, i, 3)
    program[out] = int(a < b)
    return i + 4, None

def eq(program, i, **kwargs):
    assert program[i] % 100 == 8
    a, b, out = params(program, i, 3)
    program[out] = int(a == b)
    return i + 4, None

def rbo(program, i, **kwargs):
    assert program[i] % 100 == 9
    delta, _ = params(program, i, 2)
    program.offset += delta
    # print (f"> rbo = {program.offset}" )
    return i + 2, None

operations = { 1: add, 2: mul, 3: inp, 4: out, 5: jit, 6: jif, 7: lt, 8: eq, 9: rbo }

class Program(list):
    def __init__(self, *args, **kwargs):
        super(Program, self).__init__(args[0])
        self.offset = 0

def compute(program, debug=False):
    program = Program(program)
    
    l = len(program)
    i = 0
    output_value = None
    last_output = None
    
    while i < l:
        op = program[i]
        if op == 99:
            if debug: 
                print(f">> exit {output_value}")
            return 
        
        if debug:
            print(f">> {i}, {list(enumerate(program))[i:i+5]}")
            
        if op % 100 == 3:
            input_value = yield
        else:
            input_value = None
            
        i, output_value = operations[op % 100](program, i, input_value=input_value)
        
        if output_value is not None:
            yield output_value
        
    raise Exception("Didn't terminate properly")
    
def process(listing):
    lst = list(map(int, listing.split(",")))
    lst.extend([0] * 100000)
    return lst

In [129]:
program = readfile("data/day19.txt")

In [132]:
def test_point(x, y):
    p = compute(process(program.strip()))
    assert next(p) == None
    assert p.send(x) == None
    return p.send(y)

def find(size):
    x, y = 0, 0
    while True:
        if y >= size - 1:
            if test_point(x + size - 1, y - size + 1):
                return x, y - size + 1

        i = 0
        passed = test_point(x + i, y + 1)
        while not passed and i <= 10:
            i += 1
            passed = test_point(x + i, y + 1)
        
        if passed:
            x = x + i
        y = y + 1
    
x, y = find(100)

In [133]:
x * 10000 + y

11351625